# Logistic regression results

Logistic regression was performed on the biowulf cluster. 

All raw outputs are located on biowulf. Tuned outputs are also located in biowulf. 

Data is backed up in curium but the Jupyter notebooks only exist in biowulf (currently).

## Changes

- 2024-05-17
    - Added section for KDD figures
- 2024-05-14 to 15
    - Added Llama 3

## Parameters

Run this to load the standard list of models, datas (segmentations), features, C values, and training types. 

In [54]:
models = [
    'bert_base_uncased',
    'llama_7b', 
    'llama_13b', 
    'llama3_8b', 
    'llama3_8b_instruct',
    'mental_bert', 
    'mental_longformer',
    # 'roberta'
]
    
datas = [
    'all', 
    'pt_noshort',
    'turns', 
]

features = [
    'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', # 'a8', 
    'b1', 'b2', 'b3', 
    'c1', 'c2', 'c3', 'c4', 
    'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 
    'e1', 'e2', 'e3', # 'e4', # 'e5', 
    'g1', 'g2', 
    'f1', 'f2', 'f3', 'f',
    'a', 'b', 'c', 'd', 'e', 'g', 
    'any'
]

agg_features = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'any']
outcomes = list(set(features) - set(agg_features))
outcomes.sort()

k = 4
C_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1., 5., 10., 50., 100., 500., 1000., 5000.]
trn_types = ['baseline', 'perm_trn']

Below is a basic function to return the most common subset: baseline train type and specific outcomes only. 

In [2]:
def subset_tuned(df, tt='baseline'):
    return df[
        (df['trn_type'] == tt) & 
        df['feature'].isin(outcomes)
    ]

### Paper-friendly modifications

Below are lists to help with converting labels to paper-friendly formats. 

In [ ]:
# Single-line names
model_names_1 = [
    'BERT', 
    'Llama 2-7B', 
    'Llama 2-13B', 
    'Llama 3-8B',
    'Llama 3-8B Instruct',
    'MentalBERT', 
    'MentalLongformer'
]
# Double-line names
model_names_2 = [
    'BERT', 
    'Llama 2\n7B', 
    'Llama 2\n13B', 
    'Llama 3\n8B',
    'Llama 3\n8B Instr.',
    'Mental\nBERT', 
    'Mental\nLongformer'
]
# Renaming segmentations
segmentations = {
    'all': 'Original', 
    'pt_noshort': 'Monologue', 
    'turns': 'Turns'
}

### Write paths

In [ ]:
from pathlib import Path

share_path = Path('/Datasets/impactme/results/20240515')
share_path.mkdir(exist_ok=True)

### Concatenating additional results

In [ ]:
import pandas as pd 

tuned_df = pd.read_csv('results/tuned/roc_auc-kfold.csv')

# Concatenating the tuned results with roc_auc-kfold-0515
tuned_df = pd.concat([
    tuned_df, 
    pd.read_csv('results/tuned/roc_auc-kfold-0515.csv')
])

tuned_df.to_csv(share_path / 'roc_auc-kfold.csv', index=False)
tuned_df.to_csv('results/tuned/roc_auc-kfold.csv', index=False)

## Baseline vs. permuted data

In [ ]:
# We can also save a summary df
# Reporting ROC AUC's mean across folds and its SD

summary_df = tuned_df.groupby(['model', 'data', 'feature', 'trn_type']).agg({'roc_auc': ['mean', 'std']}).reset_index()
summary_df.columns = ['model', 'data', 'feature', 'trn_type', 'roc_auc_mean', 'roc_auc_sd']
summary_df.tail(10)

#### As tables

In [ ]:
# Display the summary df with only trn_type baseline or perm_trn
base_v_perm = tuned_df[tuned_df['trn_type'].isin(['baseline', 'perm_trn'])]
# Make table wider using trn_type
base_v_perm.pivot_table(index=['model', 'data', 'feature'], columns='trn_type', values='roc_auc')

In [ ]:

# Save the summary as a CSV
summary_df.to_csv('results/tuned/roc_auc-kfold-summary.csv', index=False)
summary_df.to_csv(share_path / 'roc_auc-kfold-summary.csv', index=False)

### As bar graphs

#### Compiling all data

Results can be presented as a simple bar graph. 

These graphs are faceted by dataset. Models are on the x-axis, with the baseline and permutation test side-by-side (color-coded).

Advantages:

- Less visual clutter
- Clear demo of permutation test working

Disadvantages: 

- Loss of information across features
- Assumes that feature performance can be aggregated

In [ ]:
tuned_df['model'].unique()  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

tuned = pd.read_csv('results/tuned/roc_auc-kfold-summary.csv')
baseline = subset_tuned(tuned, 'baseline')
perm_trn = subset_tuned(tuned, 'perm_trn')

fig, axs = plt.subplots(1, 3, figsize=(15, 4))

for data, ax in zip(datas, axs):
    # Create side-by-side bar plot for baseline and perm_trn compared across models
    x = np.arange(len(models))
    width = 0.35

    heights = [baseline[baseline['model'] == model]['roc_auc_mean'].mean() for model in models]
    errbars = [baseline[baseline['model'] == model]['roc_auc_mean'].std() for model in models]
    ax.bar(x - width/2, heights, width, yerr=errbars, label='Baseline')

    heights = [perm_trn[perm_trn['model'] == model]['roc_auc_mean'].mean() for model in models]
    errbars = [perm_trn[perm_trn['model'] == model]['roc_auc_mean'].std() for model in models]
    # Manually add offset
    ax.bar(x + width/2, heights, width, yerr=errbars, label='Perm.') 

    ax.set_xticks(x)
    ax.set_xticklabels(model_names_2)
    # Make xtick_labels smaller
    ax.tick_params(axis='x', labelsize=10)
    ax.set_title(segmentations[data])

    ax.set_ylim(0.0, 1.0)


# Tight layout
plt.tight_layout()
# Add axis labels
fig.text(0.5, 0, 'Model', ha='center', va='center', fontsize=12)
fig.text(0, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=12)
# fig.suptitle('Average performance at baseline and with training label permutation', fontsize=14)

plt.legend()
plt.show()


#### Only for 'any'

Here's the bar chart with only 'any' across its four folds. We'll be taking the SD of the ROC AUCs across the outer folds as the error bars on the chart. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

tuned_raw = pd.read_csv('results/tuned/roc_auc-kfold.csv')

tuned_raw = pd.concat([
    tuned_raw,
    pd.read_csv('results/tuned/roc_auc-kfold-0515.csv')
])

baseline = tuned_raw[tuned_raw['trn_type'] == 'baseline']
perm_trn = tuned_raw[tuned_raw['trn_type'] == 'perm_trn']

fig, axs = plt.subplots(1, 3, figsize=(15, 4))

for data, ax in zip(datas, axs):
    x = np.arange(len(models))
    width = 0.35

    heights = [baseline[(baseline['model'] == model) & (baseline['feature'] == 'any')]['roc_auc'].mean() for model in models]
    errbars = [baseline[(baseline['model'] == model) & (baseline['feature'] == 'any')]['roc_auc'].std() for model in models]
    ax.bar(x - width/2, heights, width, yerr=errbars, label='Baseline')

    heights = [perm_trn[(perm_trn['model'] == model) & (perm_trn['feature'] == 'any')]['roc_auc'].mean() for model in models]
    errbars = [np.nanstd(perm_trn[(perm_trn['model'] == model) & (perm_trn['feature'] == 'any')]['roc_auc'].values) for model in models]
    ax.bar(x + width/2, heights, width, yerr=errbars, label='Perm.')

    ax.set_xticks(x)
    ax.set_xticklabels(model_names_2)
    ax.tick_params(axis='x', labelsize=10)
    ax.set_title(segmentations[data])

    ax.set_ylim(0.0, 1.0)

# Tight layout
plt.tight_layout()
fig.text(0.5, 0, 'Model', ha='center', va='center', fontsize=12)
fig.text(0, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=12)
# fig.suptitle('Average performance at baseline and with training label permutation', fontsize=14)

plt.legend()
plt.show()

## Performance of all classifiers

Faceted on segmentations, we can plot the performance of each feature for each model as a jittered scatter plot. Here, we will only look at baseline results.

The graph produced in this step is systematically jittered by domain so that they are roughly grouped vertically. 

In [ ]:
import numpy as np
   
def jitter_dots(dots):
    offsets = dots.get_offsets()
    jittered_offsets = offsets
    # only jitter in the x-direction
    jittered_offsets[:, 0] += np.random.uniform(-0.3, 0.3, offsets.shape[0])
    dots.set_offsets(jittered_offsets)

Helper variables indicate the domain and whether a feature is an aggregate. 

In [ ]:
tuned_grp = tuned.copy()
tuned_grp = tuned_grp[~tuned_grp['feature'].str.contains('any')]

# Create new column 'domain' that is the first character in 'feature' 
tuned_grp['domain'] = tuned_grp['feature'].str[0]

# Create an indicator column 'aggregate' that is True if 'feature' is one letter
tuned_grp['aggregate'] = tuned_grp['feature'].str.len() == 1

domains = tuned_grp['domain'].unique()

tuned_any_temp = tuned[tuned['feature'].str.contains('any')].copy()
tuned_any_temp['domain'] = 'any'
tuned_any_temp['aggregate'] = True

tuned_grp = pd.concat([tuned_grp, tuned_any_temp])

Below, we facet by segmentation horizontally, so that performance can be tracked horizontally across the segmentations. 

Although easier to compare segmentations, this produces a very wide graph. 

In [ ]:
fig, axs = plt.subplots(1, len(datas), figsize=(20, 4))
baseline = tuned_grp[tuned_grp['trn_type'] == 'baseline']

np.random.seed(402)

colors_list = ['blue', 'green', 'purple', 'orange', 'brown', 'pink', 'olive', 'cyan']
colors_list = colors_list[:len(domains)]
domain_colors = {domain: color for domain, color in zip(domains, colors_list)}
domain_offsets = {
    'a': -0.33,
    'b': -0.22,
    'c': -0.11,
    'd': 0.0,
    'e': 0.11,
    'f': 0.22,
    'g': 0.33,
    'any': 0.055,
}

for data, ax in zip(datas, axs):
    # Models on the x-axis, ROC AUC on the y-axis

    # Plot non-aggregate features first
    for i, row in baseline[(baseline['data'] == data) & (~baseline['aggregate'])].iterrows():
        color = domain_colors[row['domain']]
        marker = f'${row["feature"][1:]}$'
        dots = ax.scatter(row['model'], row['roc_auc_mean'], marker=marker, color=color, alpha=0.5)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']] + np.random.uniform(-0.04, 0.04, 1)[0], 0])
        dots.set_offsets(jittered_offsets)

    # Plot aggregate features last
    for i, row in baseline[(baseline['data'] == data) & (baseline['aggregate'])].iterrows():
        color = 'black'
        marker = f'${row["feature"].upper()}$'
        if row['feature'] == 'any':
            marker = 'X'
            color = 'red'
        dots = ax.scatter(row['model'], row['roc_auc_mean'], marker=marker, color=color, alpha=0.5)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']], 0])
        dots.set_offsets(jittered_offsets)

    ax.set_title(data)
    ax.set_ylim(0.45, 1.0)
    # Increase xlim to accommodate jitter
    ax.set_xlim(-0.6, len(models)-0.4)

# Create legend
handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=domain) for domain, color in zip(domains, colors_list)
]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='black', label='aggregate')]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='red', label='any')]
fig.legend(handles=handles, loc='center right')

# y-axis label
fig.text(0.1, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=12)
plt.show()

We can also try to facet vertically, at the cost of comparisons across segmentations. This will make the jitter more displaced and clear. 

In [ ]:
fig, axs = plt.subplots(len(datas), 1, figsize=(10, 10))
baseline = tuned_grp[tuned_grp['trn_type'] == 'baseline']

np.random.seed(402)

colors_list = ['blue', 'green', 'purple', 'orange', 'brown', 'pink', 'olive', 'cyan']
colors_list = colors_list[:len(domains)]
domain_colors = {domain: color for domain, color in zip(domains, colors_list)}
domain_offsets = {
    'a': -0.33,
    'b': -0.22,
    'c': -0.11,
    'd': 0.0,
    'e': 0.11,
    'f': 0.22,
    'g': 0.33,
    'any': 0.055,
}

for data, ax in zip(datas, axs):
    # Models on the x-axis, ROC AUC on the y-axis

    # Plot non-aggregate features first
    for i, row in baseline[(baseline['data'] == data) & (~baseline['aggregate'])].iterrows():
        color = domain_colors[row['domain']]
        marker = f'${row["feature"][1:]}$'
        dots = ax.scatter(row['model'], row['roc_auc_mean'], marker=marker, color=color, alpha=0.35)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']] + np.random.uniform(-0.04, 0.04, 1)[0], 0])
        dots.set_offsets(jittered_offsets)

    # Plot aggregate features last
    for i, row in baseline[(baseline['data'] == data) & (baseline['aggregate'])].iterrows():
        color = 'black'
        marker = f'${row["feature"].upper()}$'
        if row['feature'] == 'any':
            marker = 'X'
            color = 'red'
        dots = ax.scatter(row['model'], row['roc_auc_mean'], marker=marker, color=color, alpha=.6)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']], 0])
        dots.set_offsets(jittered_offsets)

    ax.set_title(segmentations[data])
    ax.set_ylim(0.45, 1.0)
    # Increase xlim to accommodate jitter
    ax.set_xlim(-0.6, len(models)-0.4)

    ax.set_xticklabels(model_names_1)

# Create legend
handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=domain.upper()) for domain, color in zip(domains, colors_list)
]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='black', label='Domain')]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='red', label='Any')]
fig.legend(handles=handles, loc='lower center', ncol=len(domains) + 2)

# y-axis label
fig.text(0.06, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=12)
fig.text(0.5, 0.06, 'Model', ha='center', va='center', fontsize=12)
# plt.tight_layout() # tends to cutoff things with the legend
plt.show()

### Final figure

In [ ]:
# Redo the above model, but now with the x-axis order being
# Llama, MentalLongformer, MentalBERT, and BERT

fig, axs = plt.subplots(len(datas), 1, figsize=(10, 10))
baseline = tuned_grp[tuned_grp['trn_type'] == 'baseline']
# Replace the model names with the paper-friendly format
baseline['model_order'] = baseline['model'].replace({
    'bert_base_uncased': 0,
    'llama_7b': 3,
    'mental_bert': 2,
    'mental_longformer': 1
})
models_ordered = ['Llama 2-7B', 'Mental\nLongformer', 'Mental\nBERT', 'BERT'][::-1]

np.random.seed(402)

colors_list = ['blue', 'green', 'purple', 'orange', 'brown', 'pink', 'olive', 'cyan']
colors_list = colors_list[:len(domains)]
domain_colors = {domain: color for domain, color in zip(domains, colors_list)}
domain_offsets = {
    'a': -0.33,
    'b': -0.22,
    'c': -0.11,
    'd': 0.0,
    'e': 0.11,
    'f': 0.22,
    'g': 0.33,
    'any': 0.055,
}

for data, ax in zip(datas, axs):
    # Models on the x-axis, ROC AUC on the y-axis

    # Plot non-aggregate features first
    for i, row in baseline[(baseline['data'] == data) & (~baseline['aggregate'])].iterrows():
        color = domain_colors[row['domain']]
        marker = f'${row["feature"][1:]}$'
        dots = ax.scatter(row['model_order'], row['roc_auc_mean'], marker=marker, color=color, alpha=0.35)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']] + np.random.uniform(-0.04, 0.04, 1)[0], 0])
        dots.set_offsets(jittered_offsets)

    # Plot aggregate features last
    for i, row in baseline[(baseline['data'] == data) & (baseline['aggregate'])].iterrows():
        color = 'black'
        marker = f'${row["feature"].upper()}$'
        if row['feature'] == 'any':
            marker = 'X'
            color = 'red'
        dots = ax.scatter(row['model_order'], row['roc_auc_mean'], marker=marker, color=color, alpha=.6)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']], 0])
        dots.set_offsets(jittered_offsets)

    ax.set_title(segmentations[data])
    ax.set_ylim(0.45, 1.0)
    # Increase xlim to accommodate jitter
    ax.set_xlim(-0.6, len(models)-0.4)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels(models_ordered)

# Create legend
handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=domain.upper()) for domain, color in zip(domains, colors_list)
]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='black', label='Domain')]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='red', label='Any')]
fig.legend(handles=handles, loc='lower center', ncol=len(domains) + 2)

# y-axis label
fig.text(0.06, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=14)
fig.text(0.5, 0.05, 'Model', ha='center', va='center', fontsize=14)
# plt.tight_layout() # tends to cutoff things with the legend
# Increase vertical spacing between subplots
plt.subplots_adjust(hspace=0.3)
plt.show()


### In Slides

In [ ]:
# Redo the above model, but now with the x-axis order being
# Llama, MentalLongformer, MentalBERT, and BERT

fig, axs = plt.subplots(len(datas), 1, figsize=(12, 7))
baseline = tuned_grp[tuned_grp['trn_type'] == 'baseline']
# Replace the model names with the paper-friendly format
baseline['model_order'] = baseline['model'].replace({
    'bert_base_uncased': 0,
    'llama_7b': 3,
    'mental_bert': 2,
    'mental_longformer': 1
})
models_ordered = ['Llama 2-7B', 'Mental\nLongformer', 'Mental\nBERT', 'BERT'][::-1]

np.random.seed(402)

colors_list = ['blue', 'green', 'purple', 'orange', 'brown', 'pink', 'olive', 'cyan']
colors_list = colors_list[:len(domains)]
domain_colors = {domain: color for domain, color in zip(domains, colors_list)}
domain_offsets = {
    'a': -0.33,
    'b': -0.22,
    'c': -0.11,
    'd': 0.0,
    'e': 0.11,
    'f': 0.22,
    'g': 0.33,
    'any': 0.055,
}

for data, ax in zip(datas, axs):
    # Models on the x-axis, ROC AUC on the y-axis

    # Plot non-aggregate features first
    for i, row in baseline[(baseline['data'] == data) & (~baseline['aggregate'])].iterrows():
        color = domain_colors[row['domain']]
        marker = f'${row["feature"][1:]}$'
        dots = ax.scatter(row['model_order'], row['roc_auc_mean'], marker=marker, color=color, alpha=0.35)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']] + np.random.uniform(-0.04, 0.04, 1)[0], 0])
        dots.set_offsets(jittered_offsets)

    # Plot aggregate features last
    for i, row in baseline[(baseline['data'] == data) & (baseline['aggregate'])].iterrows():
        color = 'black'
        marker = f'${row["feature"].upper()}$'
        if row['feature'] == 'any':
            marker = 'X'
            color = 'red'
        dots = ax.scatter(row['model_order'], row['roc_auc_mean'], marker=marker, color=color, alpha=.6)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']], 0])
        dots.set_offsets(jittered_offsets)

    ax.set_title(segmentations[data])
    ax.set_ylim(0.45, 1.0)
    # Increase xlim to accommodate jitter
    ax.set_xlim(-0.6, len(models)-0.4)
    ax.set_xticks(range(len(models)))
    ax.set_xticklabels(models_ordered)

# Create legend
handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, label=domain.upper()) for domain, color in zip(domains, colors_list)
]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='black', label='Domain')]
handles += [plt.Line2D([0], [0], marker='X', color='w', markerfacecolor='red', label='Any')]
fig.legend(handles=handles, loc='lower center', ncol=len(domains) + 2)

# y-axis label
fig.text(0.06, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=14)
fig.text(0.5, 0.05, 'Model', ha='center', va='center', fontsize=14)
# plt.tight_layout() # tends to cutoff things with the legend
# Increase vertical spacing between subplots
plt.subplots_adjust(hspace=0.3)
plt.show()


In [ ]:
# Plot different segmentations in 3 separate plots

baseline = tuned_grp[tuned_grp['trn_type'] == 'baseline']
baseline['model_order'] = baseline['model'].replace({
    'bert_base_uncased': 0,
    'llama_7b': 3,
    'mental_bert': 2,
    'mental_longformer': 1
})
models_ordered = ['Llama 2-7B', 'Mental\nLongformer', 'Mental\nBERT', 'BERT'][::-1]

np.random.seed(430)

for data in datas:
    # Create new figure
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))

    for i, row in baseline[(baseline['data'] == data) & (~baseline['aggregate'])].iterrows():
        color = domain_colors[row['domain']]
        marker = f'${row["feature"][1:]}$'
        dots = ax.scatter(row['model_order'], row['roc_auc_mean'], marker=marker, color=color, alpha=0.35)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']] + np.random.uniform(-0.04, 0.04, 1)[0], 0])
        dots.set_offsets(jittered_offsets)

    for i, row in baseline[(baseline['data'] == data) & (baseline['aggregate'])].iterrows():
        color = 'black'
        marker = f'${row["feature"].upper()}$'
        if row['feature'] == 'any':
            marker = 'X'
            color = 'red'
        dots = ax.scatter(row['model_order'], row['roc_auc_mean'], marker=marker, color=color, alpha=.6)
        offsets = dots.get_offsets()
        jittered_offsets = offsets + np.array([domain_offsets[row['domain']], 0])
        dots.set_offsets(jittered_offsets)

    ax.set_title(segmentations[data])
    ax.set_ylim(0.45, 1.0)
    ax.set_xlim(-0.6, len(models)-0.4)
    ax.set_xticks(range(len(models)))

    ax.set_xticklabels(models_ordered)
    fig.text(0.06, 0.5, 'ROC AUC', ha='center', va='center', rotation='vertical', fontsize=12)
    plt.show()
    

    

### Exchanging Llama 2-7B with Llama 3-8B

Same dimensions as previous figure. 

## Posthocs across models withn segmentations

All comparisons are done without aggregates. 


### Friedman test

We can use a Friedman test to compare the performance of the different models. Because the different datasets are for different use cases, we will make separate comparisons for each dataset without needing a multiple comparison correction. 

#### On separate folds

In [ ]:
from scipy.stats import friedmanchisquare
import pandas as pd

# Read in the tuning results
tuned_df = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold.csv'))

for data in datas:
    print(f'Friedman test for {data}')
    df = tuned_df[tuned_df['data'] == data].pivot(
        index=['feature', 'outer_fold'], columns='model', values='roc_auc'
    )[models]
    # remove rows with NaN
    df = df.dropna()
    print(friedmanchisquare(*[df[model] for model in models]), '\n')

models_llama = [
    'llama_7b', 
    'llama_13b', 
    'llama3_8b', 
    'llama3_8b_instruct'
]

for data in datas:
    print(f'Friedman test for {data}')
    df = tuned_df[tuned_df['data'] == data].pivot(
        index=['feature', 'outer_fold'], columns='model', values='roc_auc'
    )[models]
    # remove rows with NaN
    df = df.dropna()
    print(friedmanchisquare(*[df[model] for model in models_llama]), '\n')


#### Averaged across folds

We can also run the Friedman on averaged ROC AUCs. At $\alpha = 0.05$, we also receive significant results for each. 

In [ ]:
tuned_df = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold-summary.csv'))

for data in datas:
    print(f'Friedman test for {data}')
    df = tuned_df[tuned_df['data'] == data].pivot(
        index='feature', columns='model', values='roc_auc_mean'
    )[models]
    # remove rows with NaN
    df = df.dropna()
    print(friedmanchisquare(*[df[model] for model in models]), '\n')

for data in datas:
    print(f'Friedman test for {data}')
    df = tuned_df[tuned_df['data'] == data].pivot(
        index='feature', columns='model', values='roc_auc_mean'
    )[models]
    df = df.dropna()
    print(friedmanchisquare(*[df[model] for model in models_llama]), '\n')

### Nemenyi posthoc

#### Averaged across folds

In [ ]:
from scikit_posthocs import posthoc_nemenyi_friedman

# Read in the tuning results
tuned_df = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold-summary.csv'))
tuned_df['roc_auc_mean'] = tuned_df['roc_auc_mean'].astype(float)

In [ ]:
# Print means of ROC AUC for each model and dataset
for data in datas:
    print(f'Means for {data}')
    print(tuned_df[tuned_df['data'] == data].groupby('model').agg({'roc_auc_mean': ['mean', 'std']}), '\n')

In [ ]:
# Pivot tuned_df to have models as columns
tuned_alt = tuned_df.pivot(
    index=['feature', 'data'], columns='model', values='roc_auc_mean'
)
tuned_alt = pd.DataFrame(tuned_alt.to_records())

In [ ]:
import numpy as np

print('Model order:', models)

for data in datas:
    print(f'Nemenyi post-hoc test for {data}')
    df = tuned_alt[tuned_alt['data'] == data]
    x = np.array([df[model].values for model in models])
    print(posthoc_nemenyi_friedman(x.T), '\n')

#### Critical difference diagram

In [ ]:
tuned_df.head()

In [ ]:
from scikit_posthocs import posthoc_nemenyi_friedman
from scikit_posthocs import critical_difference_diagram
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(5, 10))

# Rename models to be appropriate for the figure
tuned_df['model_rename'] = tuned_df['model'].replace({
    'bert_base_uncased': 'BERT',
    'llama_7b': 'Llama 2-7B',
    'llama_13b': 'Llama 2-13B',
    'llama3_8b': 'Llama 3-8B',
    'llama3_8b': 'Llama 3-8B Instr.',
    'mental_bert': 'MentalBERT',
    'mental_longformer': 'MentalLongformer'
})

for data, ax in zip(datas, axs):
    df = tuned_df[tuned_df['data'] == data]
    cd = posthoc_nemenyi_friedman(
        df, 
        melted=True, 
        block_col='feature', 
        group_col='model_rename', 
        y_col='roc_auc_mean'
    )
    avg_rank = df.groupby('feature').roc_auc_mean.rank(pct=True).groupby(df.model_rename).mean()
    # Set axis limits
    ax.set_xlim(0.45, 0.8)
    critical_difference_diagram(
        avg_rank, 
        cd, 
        ax=ax, 
        elbow_props={'color': 'black'}
    )
    ax.set_title(segmentations[data])

### Bayesian posthoc

Bayesian posthocs are calculated on the mean over the outer $k$-folds. 

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(20240416)

# Read in the baseline and then take mean
tuned = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold-summary.csv'))

In [ ]:
n = len(models)

baycomp = {
    data: {
        'A>B': np.zeros((n, n)),
        'A=B': np.zeros((n, n)),
        'A<B': np.zeros((n, n))
    } for data in datas
}

#### Calculations

In [ ]:
import itertools
from baycomp import two_on_multiple

params = list(itertools.combinations(models, 2))
for d in datas:
    df = tuned[tuned['data'] == d]

    for m1, m2 in params:
        x = df[df['model'] == m1]['roc_auc_mean'].values
        y = df[df['model'] == m2]['roc_auc_mean'].values

        result = two_on_multiple(x, y, rope=0.01)

        # print(result)

        baycomp[d]['A>B'][models.index(m1), models.index(m2)] = result[0]
        baycomp[d]['A=B'][models.index(m1), models.index(m2)] = result[1]
        baycomp[d]['A<B'][models.index(m1), models.index(m2)] = result[2]

        # Fill in the other side of the diagonal as well
        baycomp[d]['A<B'][models.index(m2), models.index(m1)] = result[0]
        baycomp[d]['A=B'][models.index(m2), models.index(m1)] = result[1]
        baycomp[d]['A>B'][models.index(m2), models.index(m1)] = result[2]

#### Probability heatmaps

##### Original only

Our first plots are the results only from the Original segmentation. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fig, axs = plt.subplots(ncols=3, figsize=(25, 6))

ind_ = 0 
list_matrices = baycomp['all']
# Convert dictionary to list
list_matrices = [list_matrices['A>B'], list_matrices['A=B'], list_matrices['A<B']]
list_prob = ['P(A > B)', 'P(Model Equivalence)', 'P(A < B)']

for ind_ in range(len(list_matrices)):    
    corrM = np.round(list_matrices[ind_], 2)
    mask = np.zeros_like(corrM, dtype=bool)
    mask[np.triu_indices_from(corrM)] = True
    
    sns.heatmap(corrM, mask=mask, annot = True, fmt='.2f',
                xticklabels=model_names_2,
                yticklabels=model_names_2,
                vmin=0, vmax=1, center=0.5,
                ax=axs[ind_], cmap="Blues")
    axs[ind_].set_title('{}'.format(list_prob[ind_]), fontsize = 22, c='black')
    # Label axes
    axs[ind_].set_xlabel('Model B', fontsize = 16, c='black')
    axs[ind_].set_ylabel('Model A', fontsize = 16, c='black')
    
    
    ind_ = ind_ + 1
    
plt.show()

##### All comparisons, triangles

If we plot each comparison for each segmentation, we have a $n_{comps} \times n_{segments}$ set of graphs. Here, $n_{comps} = 3$. 

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(12, 12))
segmentations = ['Original', 'Monologue', 'Turns']
model_names = [
    'Llama 2-7B', 
    'Mental\nLongformer',
    'MentalBERT', 
    'BERT'
]

for i, data in enumerate(datas):
    list_matrices = baycomp[data]
    list_matrices = [list_matrices['A>B'], list_matrices['A=B'], list_matrices['A<B']]

    for ind_ in range(len(list_matrices)):    
        corrM = np.round(list_matrices[ind_], 2)
        mask = np.zeros_like(corrM, dtype=bool)
        mask[np.triu_indices_from(corrM)] = True

        sns.heatmap(
            corrM, mask=mask, 
            annot = True, fmt='.2f',
            xticklabels=model_names,
            yticklabels=model_names,
            vmin=0, vmax=1,
            cbar=False,
            ax=axs[i, ind_], cmap="Blues"
        )
        
        # Remove titles for all but the first row
        if i == 0:
            axs[i, ind_].set_title(f'{list_prob[ind_]}', fontsize = 14, c='black')

        # Remove yticks for all but the first column
        if ind_ != 0:
            axs[i, ind_].set_yticks([])
        else:
            # Rotate the yticklabels for the first plot
            plt.setp(axs[i, ind_].get_yticklabels(), rotation=0)

        # Orient xticklabels horizontally
        plt.setp(axs[i, ind_].get_xticklabels(), rotation=0)

plt.tight_layout()

# Add Model A to y-axis, Model B to x-axis
fig.text(0.5, -0.01, 'Model B', ha='center', va='center', fontsize=14)
fig.text(-0.01, 0.5, 'Model A', ha='center', va='center', rotation='vertical', fontsize=14)

# Label rows on the right axis as the segmentations
for i, data in enumerate(datas):
    fig.text(
        1.0, 0.15 + 0.33 * i, 
        segmentations[i], 
        ha='center', 
        va='center', 
        # rotation='vertical', 
        rotation=270,
        fontsize=14
    )

plt.show()

##### P(A > B) square (vertical text) (single colorbar)

We can also make modifications to use the full square and combine the graphs of $P(A < B)$ and $P(A > B)$. Essentially, we can just flip and rotate one graph to fit on the other. The $P(A = B)$ can then be inferred. 

In [ ]:
fig, axs = plt.subplots(1, 4, gridspec_kw={'width_ratios': [1, 1, 1, 0.08]}, figsize=(16, 5))
axs[0].get_shared_y_axes().join(axs[1], axs[2])

for i, data in enumerate(datas[:-1]):
    baycomp_mat = baycomp[data]['A>B']
    corrM = np.round(baycomp_mat, 2)
    mask = np.zeros_like(corrM, dtype=bool)
    # Mask the diagonal
    np.fill_diagonal(mask, True)

    sns.heatmap(
        corrM, mask=mask, 
        annot = True, fmt='.2f',
        xticklabels=model_names, yticklabels=model_names,
        vmin=0, vmax=1, cbar=False,
        ax=axs[i], 
        cmap="Blues")
    axs[i].set_title(f'{segmentations[data]}', fontsize = 15, c='black')

data = 'turns'
baycomp_mat = baycomp[data]['A>B']
corrM = np.round(baycomp_mat, 2)
mask = np.zeros_like(corrM, dtype=bool)
np.fill_diagonal(mask, True)

sns.heatmap(
    corrM, mask=mask, 
    annot = True, fmt='.2f',
    xticklabels=model_names, yticklabels=model_names,
    vmin=0, vmax=1, 
    cbar_ax=axs[3], cbar_kws={'label': 'P(A > B)'},
    ax=axs[2], 
    cmap="Blues")
axs[2].set_title(f'{segmentations[data]}', fontsize = 15, c='black')

plt.tight_layout(pad=1, w_pad=1, h_pad=3.0)
fig.text(0.5, -0.01, 'Model B', ha='center', fontsize = 14, c='black')
fig.text(-0.01, 0.5, 'Model A', va='center', rotation='vertical', fontsize = 14, c='black')

# fig.text(0.5, 1.0, 'Probability that Model A outperforms Model B', ha='center', fontsize = 17, c='black')
plt.show()

##### (In paper) P(A > B) square, one colorbar (main figure)

Below is a variation of the above figure without y-axis labels for the subplots besides the first. 

In [ ]:
# Modify the above to only have one colorbar
fig, axs = plt.subplots(1, 4, gridspec_kw={'width_ratios': [1, 1, 1, 0.08]}, figsize=(16, 5))
axs[0].get_shared_y_axes().join(axs[1], axs[2])

for i, data in enumerate(datas[:-1]):
    baycomp_mat = baycomp[data]['A>B']
    corrM = np.round(baycomp_mat, 2)
    mask = np.zeros_like(corrM, dtype=bool)
    # Mask the diagonal
    np.fill_diagonal(mask, True)

    sns.heatmap(
        corrM, mask=mask, 
        annot = True, fmt='.2f',
        xticklabels=model_names_2, yticklabels=model_names_2,
        vmin=0, vmax=1, cbar=False,
        ax=axs[i], 
        cmap="Blues")
    axs[i].set_title(f'{segmentations[data]}', fontsize = 15, c='black')

data = 'turns'
baycomp_mat = baycomp[data]['A>B']
corrM = np.round(baycomp_mat, 2)
mask = np.zeros_like(corrM, dtype=bool)
np.fill_diagonal(mask, True)

sns.heatmap(
    corrM, mask=mask, 
    annot = True, fmt='.2f',
    xticklabels=model_names_2, yticklabels=model_names_2,
    vmin=0, vmax=1, 
    cbar_ax=axs[3], cbar_kws={'label': 'P(A > B)'},
    ax=axs[2], 
    cmap="Blues")
axs[2].set_title(f'{segmentations[data]}', fontsize = 15, c='black')

# Remove yticks for all but the first plot
for ax in axs[1:-1]:
    ax.set_yticks([])

# Rotate the yticklabels for the first plot
plt.setp(axs[0].get_yticklabels(), rotation=0)

# Rotate the xticklabels for all plots
for ax in axs:
    plt.setp(ax.get_xticklabels(), rotation=0)

plt.tight_layout(pad=1, w_pad=1, h_pad=3.0)
fig.text(0.5, -0.01, 'Model B', ha='center', fontsize = 14, c='black')
fig.text(-0.01, 0.5, 'Model A', va='center', rotation='vertical', fontsize = 14, c='black')
# fig.text(0.5, 1.0, 'Probability that Model A outperforms Model B', ha='center', fontsize = 17, c='black')
plt.show()

We can supplement the above with figures showing the probability of model equivalence. 

In [ ]:
fig, axs = plt.subplots(1, 4, gridspec_kw={'width_ratios': [1, 1, 1, 0.08]}, figsize=(17, 5))
axs[0].get_shared_y_axes().join(axs[1], axs[2])

for i, data in enumerate(datas[:-1]):
    baycomp_mat = baycomp[data]['A=B']
    corrM = np.round(baycomp_mat, 2)
    mask = np.zeros_like(corrM, dtype=bool)
    # Mask upper triangle
    mask[np.triu_indices_from(corrM)] = True

    sns.heatmap(
        corrM, mask=mask, 
        annot = True, fmt='.2f',
        xticklabels=model_names_2, yticklabels=model_names_2,
        vmin=0, vmax=1, cbar=False,
        ax=axs[i], 
        cmap="Blues")
    axs[i].set_title(f'{segmentations[data]}', fontsize = 15, c='black')

data = 'turns'
baycomp_mat = baycomp[data]['A=B']
corrM = np.round(baycomp_mat, 2)
mask = np.zeros_like(corrM, dtype=bool)
mask[np.triu_indices_from(corrM)] = True

sns.heatmap(
    corrM, mask=mask, 
    annot = True, fmt='.2f',
    xticklabels=model_names_2, yticklabels=model_names_2,
    vmin=0, vmax=1, 
    cbar_ax=axs[3], cbar_kws={'label': 'P(A = B)'},
    ax=axs[2], 
    cmap="Blues")
axs[2].set_title(f'{segmentations[data]}', fontsize = 15, c='black')

# Rotate the xticklabels for all plots
for ax in axs:
    plt.setp(ax.get_xticklabels(), rotation=0)

plt.tight_layout(pad=.5, w_pad=.5, h_pad=3.0)
fig.text(0.5, -0.02, 'Model B', ha='center', fontsize = 14, c='black')
fig.text(-0.01, 0.5, 'Model A', va='center', rotation='vertical', fontsize = 14, c='black')

# fig.text(0.5, 1.0, 'Probability that Model A outperforms Model B', ha='center', fontsize = 17, c='black')
plt.show()

## Posthocs across segmentation within models

Additionally, we can compare model performance across segmentation within models (4 graphs). (Or just Llama 7B). (Specific outcomes only).

### Friedman's test

In [ ]:
from scipy.stats import friedmanchisquare

baseline = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold-summary.csv'))

for model in models:
    print(f'Friedman test for {model}')
    df = baseline[baseline['model'] == model].pivot(
        index='feature', columns='data', values='roc_auc_mean'
    )[datas]
    # remove rows with NaN
    df = df.dropna()
    print(friedmanchisquare(*[df[data] for data in datas]), '\n')

### Critical difference diagrams (Nemenyi)

In [ ]:
from scikit_posthocs import posthoc_nemenyi_friedman
from scikit_posthocs import critical_difference_diagram

baseline['roc_auc'] = baseline['roc_auc_mean'].astype(float)
baseline['data_rename'] = baseline['data'].replace(segmentations)

fig, axs = plt.subplots(len(models), 1, figsize=(5, 12))

for model, ax in zip(models, axs):
    df = baseline[baseline['model'] == model]
    cd = posthoc_nemenyi_friedman(
        df, 
        melted=True, 
        block_col='feature', 
        group_col='data_rename', 
        y_col='roc_auc'
    )
    avg_rank = df.groupby('feature').roc_auc.rank(pct=True).groupby(df.data_rename).mean()
    ax.set_xlim(0.3, 0.9)
    critical_difference_diagram(
        avg_rank, 
        cd, 
        ax=ax, 
        elbow_props={'color': 'black'}
    )
    ax.set_title(model_names_1[models.index(model)])

### Bayesian comparison test

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(20240416)

# Read in the baseline and then take mean
tuned = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold-summary.csv'))

In [ ]:
models = [
    'llama_7b',
    'mental_bert', 
    'mental_longformer',
    'bert_base_uncased'
]

datas = [
    'pt_noshort', 
    'turns', 
    'all'
]

n = len(datas)

baycomp = {
    model: {
        'A>B': np.zeros((n, n)),
        'A=B': np.zeros((n, n)),
        'A<B': np.zeros((n, n))
    } for model in models
}

In [ ]:
import itertools
from baycomp import two_on_multiple

params = list(itertools.combinations(datas, 2))

for model in models:
    df = tuned[tuned['model'] == model]

    for d1, d2 in params:
        x = df[df['data'] == d1]['roc_auc_mean'].values
        y = df[df['data'] == d2]['roc_auc_mean'].values

        result = two_on_multiple(x, y, rope=0.01)

        baycomp[model]['A>B'][datas.index(d1), datas.index(d2)] = result[0]
        baycomp[model]['A=B'][datas.index(d1), datas.index(d2)] = result[1]
        baycomp[model]['A<B'][datas.index(d1), datas.index(d2)] = result[2]

        # Fill in the other side of the diagonal as well
        baycomp[model]['A<B'][datas.index(d2), datas.index(d1)] = result[0]
        baycomp[model]['A=B'][datas.index(d2), datas.index(d1)] = result[1]
        baycomp[model]['A>B'][datas.index(d2), datas.index(d1)] = result[2]

In [ ]:
fig, axs = plt.subplots(1, 5, gridspec_kw={'width_ratios': [1, 1, 1, 1, 0.08]}, figsize=(20, 5))
axs[0].get_shared_y_axes().join(axs[1], axs[2])

data_names = ['Monologue', 'Turns', 'Original']

for i, model in enumerate(models[:-1]):
    baycomp_mat = baycomp[model]['A>B']
    corrM = np.round(baycomp_mat, 2)
    mask = np.zeros_like(corrM, dtype=bool)
    # Mask the diagonal
    np.fill_diagonal(mask, True)

    sns.heatmap(
        corrM, mask=mask, 
        annot = True, fmt='.2f',
        xticklabels=data_names, yticklabels=data_names,
        vmin=0, vmax=1, cbar=False,
        ax=axs[i], 
        cmap="Blues")
    axs[i].set_title(f'{model_names_1[i]}', fontsize = 15, c='black')


model = 'bert_base_uncased'
baycomp_mat = baycomp[model]['A>B']
corrM = np.round(baycomp_mat, 2)
mask = np.zeros_like(corrM, dtype=bool)
np.fill_diagonal(mask, True)

sns.heatmap(
    corrM, mask=mask, 
    annot = True, fmt='.2f',
    xticklabels=data_names, yticklabels=data_names,
    vmin=0, vmax=1, 
    cbar_ax=axs[4], cbar_kws={'label': 'P(A > B)'},
    ax=axs[3], 
    cmap="Blues")
axs[3].set_title(f'BERT', fontsize = 15, c='black')

plt.tight_layout(pad=1, w_pad=1, h_pad=3.0)
fig.text(0.5, -0.01, 'Segmentation B', ha='center', fontsize = 14, c='black')
fig.text(-0.01, 0.5, 'Segmentation A', va='center', rotation='vertical', fontsize = 14, c='black')

# fig.text(0.5, 1.0, 'Probability that Model A outperforms Model B', ha='center', fontsize = 17, c='black')
plt.show()

In [ ]:
# Change the above to a 2 x 2 diagram
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
data_names = ['Monologue', 'Turns', 'Original']

for i, model in enumerate(models):
    baycomp_mat = baycomp[model]['A>B']
    corrM = np.round(baycomp_mat, 2)
    mask = np.zeros_like(corrM, dtype=bool)
    # Mask the diagonal
    np.fill_diagonal(mask, True)

    sns.heatmap(
        corrM, mask=mask, 
        annot = True, fmt='.2f',
        xticklabels=data_names, yticklabels=data_names,
        vmin=0, vmax=1, cbar=False,
        ax=axs[i//2, i%2], 
        cmap="Blues")
    axs[i//2, i%2].set_title(f'{model_names_1[i]}', fontsize = 15, c='black')

plt.tight_layout(pad=1, w_pad=1, h_pad=3.0)
fig.text(0.5, -0.01, 'Segmentation B', ha='center', fontsize = 14, c='black')
fig.text(-0.01, 0.5, 'Segmentation A', va='center', rotation='vertical', fontsize = 14, c='black')


### Only Llama 7B: Nemenyi and Bayesian

For the preprint, we can produce the diagrams only for Llama 7B.

In [ ]:
fig, axs = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 2]}, figsize=(10, 4))

baseline = subset_tuned(pd.read_csv('results/tuned/roc_auc-kfold-summary.csv'))
baseline['roc_auc'] = baseline['roc_auc_mean'].astype(float)
baseline['data_rename'] = baseline['data'].replace(segmentations)

# Plot Nemenyi in first plot
df = baseline[baseline['model'] == 'llama_7b']
cd = posthoc_nemenyi_friedman(
    df, 
    melted=True, 
    block_col='feature', 
    group_col='data_rename', 
    y_col='roc_auc'
)
avg_rank = df.groupby('feature').roc_auc.rank(pct=True).groupby(df.data_rename).mean()
critical_difference_diagram(
    avg_rank, 
    cd, 
    ax=axs[0], 
    elbow_props={'color': 'black'}
)
axs[0].set_title('Nemenyi posthoc', fontsize=14)

# Plot Bayes in second plot
baycomp_mat = baycomp['llama_7b']['A>B']
corrM = np.round(baycomp_mat, 2)
mask = np.zeros_like(corrM, dtype=bool)
# Mask the diagonal
np.fill_diagonal(mask, True)

sns.heatmap(
    corrM, mask=mask, 
    annot = True, fmt='.2f',
    xticklabels=data_names, yticklabels=data_names,
    vmin=0, vmax=1, cbar=False,
    ax=axs[1], 
    cmap="Blues")
axs[1].set_title('Bayesian comparison', fontsize=14)
axs[1].set_ylabel('Segmentation A', fontsize=13, c='black')
axs[1].set_xlabel('Segmentation B', fontsize=13, c='black')

plt.tight_layout()
plt.show()

## Annotation burden

A bespoke statistic: the recall (proportion of positives labeled as positive) vs. the proportion of the transcript that needs to be reviewed, given some threshold. 

This statistic is only reporteed on the 'any' feature, as we would likely be using the model to screen for the presence of any of the specific outcomes in a text. 

In [ ]:
outputs = pd.read_csv('results/tuned/concat.csv')

In [ ]:
any_pred = outputs[
    (outputs['feature'] == 'any') &
    (outputs['trn_type'] == 'baseline')
]

### Demo: aggregated data

Like ROC, it is unclear how to best aggregate these curves over folds, and the best approach may be to plot each fold separately. However, when given enough data, such as with the `any` feature, it may be fine to work with the concatenated data to produce the curves. 

Below is a demonstration of the curves for each fold and the concatenated curve. The model used is BERT.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

fig, axs = plt.subplots(1, 3, figsize=(13, 4))
any_pred_bert = any_pred[any_pred['model'] == 'bert_base_uncased']

# USe only any_pred_bert, and plot separate lines for each outer_fold
for i, data in enumerate(datas):
    for fold in range(4):
        df = any_pred_bert[(any_pred_bert['data'] == data) & (any_pred_bert['outer_fold'] == fold)]
        y_true = df['y_true'].values
        y_prob = df['y_prob'].values
        precision, recall, _ = precision_recall_curve(y_true, y_prob)
        _, recall, thresh = precision_recall_curve(y_true, y_prob)
        pos_pred = np.sum(y_prob[:, None] > thresh, axis=0) / len(y_true)
        pos_pred = np.insert(pos_pred, 0, 1)
        axs[i].plot(pos_pred, recall, label=f'Fold {fold}')
    
    # Add the curve for all points
    df = any_pred_bert[any_pred_bert['data'] == data]
    y_true = df['y_true'].values
    y_prob = df['y_prob'].values
    precision, recall, _ = precision_recall_curve(y_true, y_prob)
    _, recall, thresh = precision_recall_curve(y_true, y_prob)
    pos_pred = np.sum(y_prob[:, None] > thresh, axis=0) / len(y_true)
    pos_pred = np.insert(pos_pred, 0, 1)
    axs[i].plot(pos_pred, recall, label='Concat')  


    # Add legend
    axs[i].legend()
    axs[i].set_title(segmentations[data])


plt.show()

### Comparison across models within segmentations

In [ ]:
from sklearn.metrics import precision_recall_curve
import numpy as np
import pandas as pd
import itertools

anno_burden = []

params = itertools.product(datas, models)
for data, model in params:
        df = any_pred[
            (any_pred['data'] == data) & 
            (any_pred['model'] == model)
        ]
        y_true = df['y_true'].values
        y_prob = df['y_prob'].values
        _, recall, thresh = precision_recall_curve(y_true, y_prob)
        pos_pred = np.sum(y_prob[:, None] > thresh, axis=0) / len(y_true)
        pos_pred = np.insert(pos_pred, 0, 1)
        anno_burden.append(
            pd.DataFrame({
                'recall': recall,
                'pos_pred': pos_pred, 
                'model': model,
                'data': data
            })
        )

anno_burden = pd.concat(anno_burden)

For each model and segmentation, we can also evaluate the proportion of text reviewed that corresponds to 50 percent and 80 percent recall.

In [ ]:
import itertools
params = itertools.product(datas, models)

target_recalls = [0.5, 0.8, 0.95]
anno_burden_intercepts = []

for i, data in enumerate(datas):
    for model in models:
        df = anno_burden[
            (anno_burden['data'] == data) & 
            (anno_burden['model'] == model)
        ]
        pos_pred = df['pos_pred'].values
        recall = df['recall'].values

        for target_recall in target_recalls:
            anno_burden_intercepts.append(
                pd.DataFrame({
                    'model': model,
                    'data': data,
                    'target_recall': [target_recall],
                    'pos_pred_need': [pos_pred[np.argmin(recall >= target_recall)]]
                })
            )

anno_burden_intercepts = pd.concat(anno_burden_intercepts)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(13, 4))

for i, data in enumerate(datas):
    for model in models:
        df = anno_burden[
            (anno_burden['data'] == data) & 
            (anno_burden['model'] == model)
        ]
        pos_pred = df['pos_pred'].values
        recall = df['recall'].values
        axs[i].plot(pos_pred, recall, label=model_names_2[models.index(model)])
    
    for target_recall in target_recalls:
        # Get mean pos_pred_need for all models
        pos_pred_need = anno_burden_intercepts[
            (anno_burden_intercepts['data'] == data) & 
            (anno_burden_intercepts['target_recall'] == target_recall)
        ]['pos_pred_need'].mean()

        if target_recall == 0.95:
            print(pos_pred_need)

        # Draw horizontal lines for recall values
        axs[i].hlines(
            target_recall, 
            -0.1, pos_pred_need, 
            color='black', linestyle='--', alpha=0.5
        )
        
        # Draw vertical lines at the average of pos_pred_need for all models
        axs[i].vlines(
            pos_pred_need, 
            -0.1, target_recall, 
            color='black', linestyle='--', alpha=0.5
        )

    axs[i].set_xlim(-0.05, 1.05)
    axs[i].set_ylim(-0.05, 1.05)

    axs[i].set_title(segmentations[data])
    axs[i].legend()


fig.text(0.5, 0.0, 'Proportion of text reviewed', ha='center', fontsize=12)
fig.text(0.09, 0.5, 'Recall', va='center', rotation='vertical', fontsize=12)

plt.show()

In [ ]:
anno_burden_intercepts

In [ ]:
intercepts_wide = anno_burden_intercepts.copy()
intercepts_wide = intercepts_wide.pivot(
    index='model', columns=['data', 'target_recall'], values='pos_pred_need'
)
# Convert to percentages with 2 decimal points
intercepts_wide = intercepts_wide.round(3)
# Print to LaTeX
print(intercepts_wide.to_latex())


## Tables

Below are LaTeX tables corresponding to relevant data. 

### ROC AUC over all folds

In [ ]:
tuned = pd.read_csv('results/tuned/roc_auc-kfold.csv')
del tuned['best_C']

In [ ]:
# Add new rows corresponding to the averages
tuned_avg = tuned.groupby(['data', 'model', 'trn_type', 'feature']).mean().reset_index()
tuned_avg['outer_fold'] = 'Mean'
# Row bind to tuned
tuned = pd.concat([tuned, tuned_avg])
tuned.tail()

In [ ]:
# Rename models
tuned['model'] = tuned['model'].replace({
    'bert_base_uncased': 'BERT',
    'llama_7b': 'Llama 7B',
    'mental_bert': 'MentalBERT',
    'mental_longformer': 'MentalLongformer'
})

# Rename features (to upper)
tuned['feature'] = tuned['feature'].str.upper()

# Reset the index
tuned.reset_index(drop=True, inplace=True)

#### All values and folds

In [ ]:
# For some reason, doing this in one step throws an IndexError
# Merging two separate data frames is fine
baseline = tuned[tuned['trn_type'] == 'baseline']
baseline_wide = baseline.pivot_table(
    index=['data', 'model', 'feature'], columns='outer_fold', values='roc_auc'
).to_records()
perm_wide = tuned[tuned['trn_type'] == 'perm_trn'].pivot_table(
    index=['data', 'model', 'feature'], columns='outer_fold', values='roc_auc'
).to_records()

# Merge the two wide dataframes
wide = pd.merge(
    pd.DataFrame(baseline_wide), 
    pd.DataFrame(perm_wide), 
    on=['data', 'model', 'feature']
)

In [ ]:
# Round to 3 decimal places, print to LaTeX
wide_sf3 = wide.round(3)
print(wide_sf3.to_latex(index=False))

#### Only the averages

In [ ]:
tuned_avg = pd.read_csv('results/tuned/roc_auc-kfold-summary.csv')
tuned_avg = tuned_avg[tuned_avg['trn_type'] == 'baseline']
del tuned_avg['trn_type']
tuned_avg['model'] = tuned_avg['model'].replace({
    'bert_base_uncased': 'BERT',
    'llama_7b': 'Llama 7B',
    'mental_bert': 'MentalBERT',
    'mental_longformer': 'MentalLongformer'
})
tuned_avg['feature'] = tuned_avg['feature'].str.upper()
tuned_avg_wide = tuned_avg.pivot_table(
    index=['feature'], 
    columns=['data', 'model'], 
    values='roc_auc_mean'
)

In [ ]:
# Round and print
print(tuned_avg_wide.round(2).to_latex())

#### As rankings

In [ ]:
# Convert columns of tuned_avg_wide to rankings
tuned_rank = tuned_avg_wide.copy()
tuned_rank = tuned_rank[~tuned_rank.index.isin(aggs)]
tuned_rank = tuned_rank.rank(axis=0, method='min', ascending=False)
# Remove rows where feature is in agg_features
aggs = [f.upper() for f in agg_features]
# Convert to integers
tuned_rank = tuned_rank.astype(int)

In [ ]:
# Create column that is the average of the row
tuned_rank['avg_rank'] = tuned_rank.mean(axis=1).round(2)
# Sort by ascending avg_rank
tuned_rank = tuned_rank.sort_values('avg_rank')
print(tuned_rank.to_latex())